Develop a Small LSTM Recurrent Neural Network

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


## Reading dataset

In [ ]:
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/to-download/the end.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


In [ ]:
raw_text

'end of project gutenberg\'s alice\'s adventures in wonderland, by lewis carroll\n\n*** end of this project gutenberg ebook alice\'s adventures in wonderland ***\n\n***** this file should be named 11.txt or 11.zip *****\nthis and all associated files of various formats will be found in:\n        http://www.gutenberg.org/1/11/\n\n\n\nupdated editions will replace the previous one--the old editions\nwill be renamed.\n\ncreating the works from public domain print editions means that no\none owns a united states copyright in these works, so the foundation\n(and you!) can copy and distribute it in the united states without\npermission and without paying copyright royalties.  special rules,\nset forth in the general terms of use part of this license, apply to\ncopying and distributing project gutenberg-tm electronic works to\nprotect the project gutenberg-tm concept and trademark.  project\ngutenberg is a registered trademark, and may not be used if you\ncharge for the ebooks, unless you rec

## Unique Character and char to int

In [ ]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '$': 4,
 '%': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '/': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 '@': 25,
 'a': 26,
 'b': 27,
 'c': 28,
 'd': 29,
 'e': 30,
 'f': 31,
 'g': 32,
 'h': 33,
 'i': 34,
 'j': 35,
 'k': 36,
 'l': 37,
 'm': 38,
 'n': 39,
 'o': 40,
 'p': 41,
 'q': 42,
 'r': 43,
 's': 44,
 't': 45,
 'u': 46,
 'v': 47,
 'w': 48,
 'x': 49,
 'y': 50,
 'z': 51}

## Total Character and Vocab

In [ ]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


Total Characters:  18747
Total Vocab:  52


## Feature and Label data

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  18647


## Reshape

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

## Normalize

In [ ]:
# normalize
X = X / float(n_vocab)

## one-hot-encoding

In [ ]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Model

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 52)                13364     
Total params: 277,556
Trainable params: 277,556
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss= 'categorical_crossentropy' , 
              optimizer= 'adam' )

## Checkpoint

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


## Fit Model

In [ ]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
18647/18647 [==============================] - 117s 6ms/step - loss: 3.1471

Epoch 00001: loss improved from inf to 3.14707, saving model to weights-improvement-01-3.1471.hdf5
Epoch 2/20
18647/18647 [==============================] - 117s 6ms/step - loss: 3.0872

Epoch 00002: loss improved from 3.14707 to 3.08718, saving model to weights-improvement-02-3.0872.hdf5
Epoch 3/20
18647/18647 [==============================] - 117s 6ms/step - loss: 3.0832

Epoch 00003: loss improved from 3.08718 to 3.08320, saving model to weights-improvement-03-3.0832.hdf5
Epoch 4/20
18647/18647 [==============================] - 118s 6ms/step - loss: 3.0682

Epoch 00004: loss improved from 3.08320 to 3.06821, saving model to weights-improvement-04-3.0682.hdf5
Epoch 5/20
18647/18647 [==============================] - 117s 6ms/step - loss: 3.0470

Epoch 00005: loss improved from 3.06821 to 3.04697, saving model to weights-improvement-05-3.0470.hdf5
Epoch 6/20
18647/18647 [=========================

Generating Text with an LSTM Network

In [ ]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


## Reading Dataset

In [ ]:
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/to-download/the end.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

## Unique Character and int-to char, char-to-int

In [ ]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

## Total char and vocabulary

In [ ]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  18747
Total Vocab:  52


## Feature and Label data

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  18647


## Reshape

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

## Normalize

In [ ]:
# normalize
X = X / float(n_vocab)

## One Hot Encoding

In [ ]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Model

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 52)                13364     
Total params: 277,556
Trainable params: 277,556
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam')


## Weight of the Model

In [ ]:
# load the network weights
filename = "/content/weights-improvement-20-2.3620.hdf5"
model.load_weights(filename)


## Random seed

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" gutenberg-tm work, and (c) any defect you cause.


section  2.  information about the mission of pro "


## Generate character

In [ ]:
# generate characters
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")

 pro ie the  ao ao                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Larger LSTM Recurrent Neural Network

In [ ]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/to-download/the end.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [ ]:
raw_text

## Unique char and char-to-int

In [ ]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '$': 4,
 '%': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '/': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 '@': 25,
 'a': 26,
 'b': 27,
 'c': 28,
 'd': 29,
 'e': 30,
 'f': 31,
 'g': 32,
 'h': 33,
 'i': 34,
 'j': 35,
 'k': 36,
 'l': 37,
 'm': 38,
 'n': 39,
 'o': 40,
 'p': 41,
 'q': 42,
 'r': 43,
 's': 44,
 't': 45,
 'u': 46,
 'v': 47,
 'w': 48,
 'x': 49,
 'y': 50,
 'z': 51}

## summary of the data

In [ ]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  18747
Total Vocab:  52


## Feature and Label data

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  18647


## Reshape

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

## Normalize

In [ ]:
# normalize
X = X / float(n_vocab)

## One-hot-encoding

In [ ]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Model

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation= 'softmax' ))


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 52)                13364     
Total params: 802,868
Trainable params: 802,868
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss= 'categorical_crossentropy', 
              optimizer= 'adam')

## callback

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Model Fit

In [72]:
# fit the model
model.fit(X, y, epochs=3, batch_size=64, callbacks=callbacks_list)

Epoch 1/3
18647/18647 [==============================] - 373s 20ms/step - loss: 3.0990

Epoch 00001: loss improved from inf to 3.09897, saving model to weights-improvement-01-3.0990-bigger.hdf5
Epoch 2/3
18647/18647 [==============================] - 375s 20ms/step - loss: 3.0790

Epoch 00002: loss improved from 3.09897 to 3.07903, saving model to weights-improvement-02-3.0790-bigger.hdf5
Epoch 3/3
18647/18647 [==============================] - 376s 20ms/step - loss: 3.0042

Epoch 00003: loss improved from 3.07903 to 3.00423, saving model to weights-improvement-03-3.0042-bigger.hdf5


In [73]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


## Reading data

In [74]:
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/to-download/the end.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

## Unique char and char-to-int, int-to-char

In [75]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

## summary of the data

In [76]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  18747
Total Vocab:  52


## Feature and Label data

In [77]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  18647


## Reshape

In [78]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))


## Normalize


In [79]:
# normalize
X = X / float(n_vocab)

## one-hot-encode

In [80]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Model

In [81]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))


In [82]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 52)                13364     
Total params: 802,868
Trainable params: 802,868
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

## Weight

In [84]:
# load the network weights
filename = "weights-improvement-03-3.0042-bigger.hdf5"
model.load_weights(filename)


## Random seed

In [85]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" eriodic tax
     returns.  royalty payments should be clearly marked as such and
     sent to the pr "


## Generate character

In [86]:
# generate characters
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")


 to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to 